In [39]:
import json

from wikifier import CallWikifier
import pandas as pd
import json
df = pd.read_csv('data/FoodBaseCurated.csv')

First we get the annotated text from wikifier

In [89]:
data = []
for i in range(5):
     response = CallWikifier(df["Full_text"].unique()[i])
     annotations = response["annotations"]
     for annotation in annotations:
        support = annotation["support"]
        indices = []
        for item in support:
            wFrom = item["wFrom"] + 1 #add 1 because the annotations in the foodbase corpus start at different index
            length = len(annotation['title'])
            indices.append((wFrom,length))
        row = [df["DocumentID"].unique()[i],annotation['title'],annotation["dbPediaIri"],indices]
        data.append(row)

wiki_df = pd.DataFrame(data, columns=['document_id','annotation_text','dbPediaIri','locations'])
wiki_df.to_csv("data/WikifierResponses.csv")
wiki_df.head()

,document_id,annotation_text,dbPediaIri,locations
0,0recipe1006,Cream cheese,http://dbpedia.org/resource/Cream_cheese,"[(3, 12)]"
1,0recipe1006,Cheese,http://dbpedia.org/resource/Cheese,"[(4, 6), (67, 6), (78, 6)]"
2,0recipe1006,Beef,http://dbpedia.org/resource/Beef,"[(6, 4)]"
3,0recipe1006,Olive,http://dbpedia.org/resource/Olive,"[(8, 5)]"
4,0recipe1006,Onion,http://dbpedia.org/resource/Onion,"[(10, 5)]"


Now we count the number of TPs, FNs and FPs:

In [90]:
TP = 0
FN = 0
FP = 0
total_annotations = 0
for recipeid in wiki_df["document_id"].unique():
    recp_wiki_df = wiki_df[wiki_df['document_id'] == recipeid]
    recp_df = df[df['DocumentID']== recipeid]
    total_annotations += len(recp_df)
    for i in range(len(recp_wiki_df)):
        locations = recp_wiki_df["locations"].iloc[i]
        for location in locations:
            for i in range(len(recp_df)):
                if location[0] == recp_df['offset'].iloc[i] and location[1] == recp_df['length'].iloc[i]:
                    TP += 1
            FP += 1
FP = FP - TP
FN = total_annotations - TP
print(f'True Positives: {TP} False Negatives: {FN}, False Positives: {FP}')

True Positives: 26 False Negatives: 40, False Positives: 82
